In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import explained_variance_score as exv, mean_absolute_error, mean_squared_error, r2_score#, mean_absolute_percentage_error as mape
from sklearn.tree import DecisionTreeRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from autokeras import StructuredDataRegressor
from keras.models import load_model
#import h2o
#from h2o.automl import H2OAutoML
#from h2o.estimators.glm import H2OGeneralizedLinearEstimator
import tkinter as tk
from tkinter import filedialog
import os
#import pywedge as pw
from sklearn.model_selection import KFold

from IPython.display import clear_output

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.ensemble import AdaBoostRegressor as ABR

import xgboost as xgb

from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNetCV

from sklearn.svm import SVR

from sklearn import neighbors

from sklearn.metrics import mean_squared_error 

from math import sqrt

from keras.callbacks import EarlyStopping

from datetime import date

#%matplotlib inline

from sklearn.metrics import mean_absolute_percentage_error as mape

import statistics

import time


from sklearn import linear_model

from sklearn.linear_model import HuberRegressor

from sklearn.linear_model import OrthogonalMatchingPursuit

from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

from sklearn.linear_model import PassiveAggressiveRegressor

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

from statsmodels.stats.outliers_influence import variance_inflation_factor


from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

In [ ]:
# Variáveis

# Conjunto de Dados
dominio = 'Campo'
alvo = 'Campo'

global method
method = 'Vazio'

In [ ]:
# Função de Cálculo de Métricas dos Modelos de Regressão

def metricas(y_test, y_pred):
    global R2
    R2 = r2_score(y_test,y_pred)
    global MSE
    MSE = mean_squared_error(y_test,y_pred)
    global RMSE
    RMSE = MSE**0.5
    global MAE
    MAE = mean_absolute_error(y_test,y_pred)
    global MAPE
    MAPE = mape(y_test, y_pred)
    #MAPE = np.mean(np.abs(percentage_error(np.asarray(y_test), np.asarray(y_pred)))) * 100
    #y_test, y_pred = np.array(y_test), np.array(y_pred)
    #MAPE =  np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print('Method: ', method)
    print("R2 = ", R2)
    print("MSE = ", MSE)
    print("RMSE = ", RMSE)
    print("MAE = ", MAE)
    print("MAPE = ", MAPE)

In [ ]:
# Abre arquivo com base de dados
root = tk.Tk()
#root.withdraw() 
arquivo = filedialog.askopenfilename()
root.update()
root.destroy()
print(arquivo)

In [ ]:
# Pegando o nome do arquivo
nome_arquivo = next(iter(arquivo))
nome_arquivo = str(arquivo)
nome_arquivo

In [ ]:
# Lê arquivo e ajusta nome da coluna: pode ser formato Xls ou csv

if nome_arquivo.split('.')[1] == 'xlsx' or nome_arquivo.split('.')[1] == 'xls':
    dados = pd.read_excel(nome_arquivo)
    #dados.to_csv(str(nome_arquivo.split('.')[0] + '.csv'))

elif nome_arquivo.split('.')[1] == 'csv':
    dados = pd.read_csv(nome_arquivo, sep=',')

else:
    print('Formato de arquivo inválido! Utilize arquivo nos formatos .xlsx, .xls ou .csv')


dados.columns = map(lambda x: str(x).upper(), dados.columns)

dados.head(5)

In [ ]:
# Ajustando nome de colunas
dados.rename(columns = {'B1': 'BAND_1',
                                 'B2': 'BAND_2',
                                 'B3': 'BAND_3',
                                 'B4': 'BAND_4',
                                 'B5': 'BAND_5',
                                 'B6': 'BAND_6',
                                 'B7': 'BAND_7',
                                 'B8': 'BAND_8',
                                 'B8A': 'BAND_8A',
                                 'B9': 'BAND_9',
                                 'B11': 'BAND_11',
                                 'B12': 'BAND_12',
                        'NUVEM_SOMBRA': 'NUVEM_SOMB',
                        'CONDUTIVIDADE': 'CONDUTIV'
                                 },
                      inplace=True)

In [ ]:
# CALCULO DE ÍNDICES ESCECTRAIS PARA COMPOR 12 (bandas) VARIÁVEIS DE ENTRADA

In [ ]:
# NDWI - Cálculo
# Sentinel-2 NDWI = (B03 - B08) / (B03 + B08)
dados['NDWI'] = (dados['BAND_3'] - dados['BAND_8']) / (dados['BAND_3'] + dados['BAND_8'])

# -----------------------

# MNDWI
# Sentinel-2 MNDWI = (B03 - B11) / (B03 + B11)
dados['M_NDWI'] = (dados['BAND_3'] - dados['BAND_11']) / (dados['BAND_3'] + dados['BAND_11'])

# -----------------------

# SWI - Cálculo
# Sentinel-2 SWI = (B05 - B11) / (B05 + B11)
dados['SWI'] = (dados['BAND_5'] - dados['BAND_11']) / (dados['BAND_5'] + dados['BAND_11'])

# -----------------------

# MCARI (Modified Chlorophyll Absorption in Reflectance Index)
##// Modified Chlorophyll Absorption in Reflectance Index   (abbrv. MCARI)
##// General formula: ((700nm - 670nm) - 0.2 * (700nm - 550nm)) * (700nm /670nm)
##// URL https://www.indexdatabase.de/db/si-single.php?sensor_id=96&rsindex_id=41
#let index = ((B05 - B04) - 0.2 * (B05 - B03)) * (B05 / B04);
dados['MCARI'] = ((dados['BAND_5'] - dados['BAND_4']) - 0.2 * (dados['BAND_5'] - dados['BAND_3'])) * (dados['BAND_5'] / dados['BAND_4'])

# -----------------------

# NDVI
# Sentinel-2 MDVI = (B08 - B04) / (B08 + B04)
dados['NDVI'] = (dados['BAND_8'] - dados['BAND_4']) / (dados['BAND_8'] + dados['BAND_4'])

# -----------------------

# NBSI
# Non-Binary Snow Index for Multi-Component Surfaces
#NBSI = ((sample.B11 + sample.B04) - (sample.B08 + sample.B02)) / ((sample.B11 + sample.B04) + (sample.B08 + sample.B02))
dados['NBSI'] = ((dados['BAND_11'] + dados['BAND_4']) - (dados['BAND_8'] + dados['BAND_2'])) / ((dados['BAND_11'] + dados['BAND_4']) + (dados['BAND_8'] + dados['BAND_2']))

# ---------------------------

#Aquatic Plants and Algae Custom Script Detector (APA Script)
##indices to apply a mask to water bodies
## moisture = (B8A-B11)/(B8A+B11); 
## NDWI = (B03 - B08)/(B03 + B08);
## water_bodies = (NDWI-moisture)/(NDWI+moisture);
dados['MOISTURE'] = (dados['BAND_8A'] - dados['BAND_11']) / (dados['BAND_8A'] + dados['BAND_11'])
dados['WATER_BODIES'] = (dados['NDWI'] - dados['MOISTURE']) / (dados['NDWI'] + dados['MOISTURE'])

# ----------------------

#indices to identify water plants and algae

# water_plants = (B05 - B04)/(B05 + B04);
# NIR2 = B04 + (B11 - B04)*((832,8 - 664,6)/(1613,7 - 664,6));
# FAI = B08 - NIR2;
dados['WATER_PLANTS'] = (dados['BAND_5'] - dados['BAND_4']) / (dados['BAND_5'] + dados['BAND_4'])

# --------------------

dados.head()

In [ ]:
# Verificação de Nuvens

In [ ]:
# Dados Relacionado à Probabilidade de Nuvens
#dados['MSK_CLDPRB'].value_counts(1) # Percentual

In [ ]:
# Dados Relacionado à Probabilidade de Nuvens
#dados['QA60'].value_counts(1)

In [ ]:
#dados[(dados['MSK_CLDPRB']<=0.0) & (dados['QA60']<=0)]
#dados[(dados['MSK_CLDPRB']<=0.0) | (dados['QA60']<=0)]
#dados[(dados['MSK_CLDPRB']>0) & (dados['QA60']==0)]

In [ ]:
#Filtros Limite de Valores Relacionados a Nuvens

print('Quer utilizar TODOS os DADOS disponíveis (SEM FILTRO DE NUVENS?)')
print('Digite 1 para SIM ou 0 para NÃO')
usar_todos_dados = int(input())

if usar_todos_dados==0:
    #print('Digite o Valor Limite da Probabilidade de Nuvem Aceitável (Valor Real. Exemplo: 10.00)')
    #valor_limite_probabilidade = float(input())

    #print('Intevelo de Dias entre a Passagem do Satélite e a Data de Coleta (inteiro)')
    #valor_QA60 = 1024.00

    #dados_filtrado = dados[(dados['MSK_CLDPRB']<=valor_limite_probabilidade) & (dados['QA60']<valor_limite_probabilidade)]
    #dados_filtrado = dados[(dados['MSK_CLDPRB']<=0.0) & (dados['QA60']<1024)]
    # dados_filtrado = dados[(dados['MSK_CLDPRB']<=0.0) & (dados['QA60']<=0)]
    dados_filtrado = dados[(dados['MSK_CLDPRB']<=0.0) & (dados['QA60']<=0) & (dados['NUVEM_SOMB'] != 'CLOUD')]
else:    
    dados_filtrado = dados.copy()


#dados_filtrado.head()
print(f'*Quantidade de Dados Disponíveis Após Aplicação do Filtro de Nuvens: {dados_filtrado.shape[0]}')

In [ ]:
#dados[(dados['MSK_CLDPRB']<=0.0) | (dados['QA60']<=0)]
dados[(dados['MSK_CLDPRB']<=0.0) & (dados['QA60']<=0) & (dados['NUVEM_SOMB'] != 'CLOUD')]

In [ ]:
# Selecionando a Variável / Parâmetro de Interesse

In [ ]:
lista_colunas = list(dados_filtrado.columns)
print(dados_filtrado.columns)

In [ ]:
lista_colunas_variaveis = list(dados_filtrado.columns[6:13])
lista_colunas_variaveis

In [ ]:
print('Selecione a variável de interesse: ')
opcoes = ["TURBIDEZ", "CHL", "SECCHI", "PH", "OD", "CONDUTIVID", "ORP"]

for i in range(len(opcoes)):
    print(str(i+1) + ":", opcoes[i])

# Loop para verificação da opção selecionada
inp = int(input("Digite o número correspondente à variável: "))
if inp in range(1,8):
    inp = opcoes[inp-1]
    print('Variável selecionada: ', inp)
else:
    print("Entrada inválida!")

variavel_interesse = str(inp)
variavel_interesse

In [ ]:
# Colunas de variáveis a serem removidas
lista_colunas_variaveis.remove(str(variavel_interesse))
lista_colunas_variaveis

In [ ]:
dados_filtrado.drop(lista_colunas_variaveis, axis=1, inplace = True)
dados_filtrado.head()

In [ ]:
# Apagando colunas desnecessárias
#del dados['UNNAMED: 0']
#del dados['GEOMETRY']

In [ ]:
# Ajustando nome de colunas
dados_filtrado.rename(columns = {'B1': 'BAND_1',
                                 'B2': 'BAND_2',
                                 'B3': 'BAND_3',
                                 'B4': 'BAND_4',
                                 'B5': 'BAND_5',
                                 'B6': 'BAND_6',
                                 'B7': 'BAND_7',
                                 'B8': 'BAND_8',
                                 'B8A': 'BAND_8A',
                                 'B9': 'BAND_9',
                                 'B11': 'BAND_11',
                                 'B12': 'BAND_12',
                                 variavel_interesse: 'VALUE'},
                      inplace=True)

In [ ]:
# Ajustar valores de Dado de Campo = Secchi: converter de cm para m
if variavel_interesse == 'SECCHI':
    dados_filtrado['VALUE'] = dados_filtrado['VALUE']/100.00
dados_filtrado.head(3)

In [ ]:
# Seleção Valores dos Parâmetros Limites da Variável de Interesse

In [ ]:
#Filtros Limite de Valor de Parâmetros e Diferenças de Datas (em dias)

print('Quer utilizar todos os dados disponíveis?')
print('Digite 1 para SIM ou 0 para NÃO')
usar_todos_dados = int(input())

if usar_todos_dados==0:
    print('Digite o valor limite do parâmetro (inteiro)')
    valor_limite = int(input())

    print('Intervalo de Dias entre a Passagem do Satélite e a Data de Coleta (inteiro)')
    diferenca_dias = int(input())

    dados_filtrado = dados_filtrado[(dados_filtrado['VALUE']<=valor_limite) & (dados_filtrado['DIFERENCA_DIAS']<=diferenca_dias)]
else:    
    dados_filtrado = dados_filtrado.copy()
    valor_limite = 'Sem Limitacao'
    diferenca_dias = 'Sem Limitacao'


#dados_filtrado.head()
print(f'*Quantidade de Dados Disponíveis Após Aplicação do Filtro de Valor Limite da Variável e Diferença entre Datas: {dados_filtrado.shape[0]}')

In [ ]:
dados_filtrado.columns

In [ ]:
# Removendo algumas colunas desnecessárias
del(dados_filtrado['UNNAMED: 0'])
del(dados_filtrado['REGIOES'])
del(dados_filtrado['ESTACAO'])
del(dados_filtrado['DATA'])
del(dados_filtrado['CAMPANHA'])
del(dados_filtrado['HORA'])
del(dados_filtrado['DATA_IMAGE'])

del(dados_filtrado['MSK_CLDPRB'])
del(dados_filtrado['NUVEM_SOMB'])
del(dados_filtrado['GEOMETRY'])
del(dados_filtrado['DIFERENCA_DIAS'])
del(dados_filtrado['QA60'])

In [ ]:
dados_filtrado.head(3)

In [ ]:
dados_filtrado.info()

In [ ]:
# Verificação de valores nulos
dados_filtrado.isnull().sum()

In [ ]:
# Excluir e verificar valores nulos
dados_filtrado.dropna(inplace = True, axis=0)
#dados.isnull().sum()
dados_filtrado.info()

In [ ]:
dados_filtrado['VALUE'].describe()

In [ ]:
# Valores iguais a Zero
dados_filtrado[dados_filtrado['VALUE'] == 0]

In [ ]:
# Remover Valores Menores ou Iguais a Zero
dados_filtrado = dados_filtrado[dados_filtrado['VALUE'] > 0]

In [ ]:
dados_filtrado['VALUE'].describe()

In [ ]:
dados_filtrado.info()

In [ ]:
dados_filtrado.describe()

In [ ]:
# Conferindo as bandas restantes
dados_filtrado.columns

In [ ]:
# Verificando a Distribuição da Variável de Interesse
x = 'VALUE'
fig, ax = plt.subplots(nrows=1, ncols=2,  sharex=False, sharey=False)
fig.suptitle(x, fontsize=20)
### distribution
ax[0].title.set_text('distribution')
variable = dados_filtrado[x].fillna(dados_filtrado[x].mean())
breaks = np.quantile(variable, q=np.linspace(0, 1, 11))
variable = variable[ (variable > breaks[0]) & (variable < 
                    breaks[10]) ]
sns.distplot(variable, hist=True, kde=True, kde_kws={"shade": True}, ax=ax[0])
des = dados_filtrado[x].describe()
ax[0].axvline(des["25%"], ls='--')
ax[0].axvline(des["mean"], ls='--')
ax[0].axvline(des["75%"], ls='--')
ax[0].grid(True)
des = round(des, 2).apply(lambda x: str(x))
box = '\n'.join(("min: "+des["min"], "25%: "+des["25%"], "mean: "+des["mean"], "75%: "+des["75%"], "max: "+des["max"]))
ax[0].text(0.95, 0.95, box, transform=ax[0].transAxes, fontsize=10, va='top', ha="right", bbox=dict(boxstyle='round', facecolor='white', alpha=1))
### boxplot 
ax[1].title.set_text('outliers (log scale)')
tmp_dtf = pd.DataFrame(dados_filtrado[x])
tmp_dtf[x] = np.log(dados_filtrado[x])
tmp_dtf.boxplot(column=x, ax=ax[1])
plt.show()

In [ ]:
# Verificação da Relação entre as Variáveis

In [ ]:
# Matriz de Correlação
sns.heatmap(dados_filtrado.corr(), annot=True) #annot mostra os coeficientes da matriz

In [ ]:
# Matriz de Correlação
#dados_filtrado.corr()

In [ ]:
# Verificação de Linearidade
#sns.pairplot(dados_filtrado, diag_kind="kde")

In [ ]:
dados_filtrado.info()

In [ ]:
# Verifidando por outro método

IQR_BAND_1 = dados_filtrado.BAND_1.quantile(0.75)-dados_filtrado.BAND_1.quantile(0.25) 
lower_bridge_BAND_1 = dados_filtrado['BAND_1'].quantile(0.25)-(IQR_BAND_1*1.5) 
upper_bridge_BAND_1 = dados_filtrado['BAND_1'].quantile(0.75)+(IQR_BAND_1*1.5)
lower_bridge_BAND_1, upper_bridge_BAND_1

IQR_BAND_2 = dados_filtrado.BAND_2.quantile(0.75)-dados_filtrado.BAND_2.quantile(0.25) 
lower_bridge_BAND_2 = dados_filtrado['BAND_2'].quantile(0.25)-(IQR_BAND_2*1.5) 
upper_bridge_BAND_2 = dados_filtrado['BAND_2'].quantile(0.75)+(IQR_BAND_2*1.5)
lower_bridge_BAND_2, upper_bridge_BAND_2

IQR_BAND_3 = dados_filtrado.BAND_3.quantile(0.75)-dados_filtrado.BAND_3.quantile(0.25) 
lower_bridge_BAND_3 = dados_filtrado['BAND_3'].quantile(0.25)-(IQR_BAND_3*1.5) 
upper_bridge_BAND_3 = dados_filtrado['BAND_3'].quantile(0.75)+(IQR_BAND_3*1.5)
lower_bridge_BAND_3, upper_bridge_BAND_3

IQR_BAND_4 = dados_filtrado.BAND_4.quantile(0.75)-dados_filtrado.BAND_4.quantile(0.25) 
lower_bridge_BAND_4 = dados_filtrado['BAND_4'].quantile(0.25)-(IQR_BAND_4*1.5) 
upper_bridge_BAND_4 = dados_filtrado['BAND_4'].quantile(0.75)+(IQR_BAND_4*1.5)
lower_bridge_BAND_4, upper_bridge_BAND_4

IQR_BAND_5 = dados_filtrado.BAND_5.quantile(0.75)-dados_filtrado.BAND_5.quantile(0.25) 
lower_bridge_BAND_5 = dados_filtrado['BAND_5'].quantile(0.25)-(IQR_BAND_5*1.5) 
upper_bridge_BAND_5 = dados_filtrado['BAND_5'].quantile(0.75)+(IQR_BAND_5*1.5)
lower_bridge_BAND_5, upper_bridge_BAND_5

IQR_BAND_6 = dados_filtrado.BAND_6.quantile(0.75)-dados_filtrado.BAND_6.quantile(0.25) 
lower_bridge_BAND_6 = dados_filtrado['BAND_6'].quantile(0.25)-(IQR_BAND_6*1.5) 
upper_bridge_BAND_6 = dados_filtrado['BAND_6'].quantile(0.75)+(IQR_BAND_6*1.5)
lower_bridge_BAND_6, upper_bridge_BAND_6

IQR_BAND_7 = dados_filtrado.BAND_7.quantile(0.75)-dados_filtrado.BAND_7.quantile(0.25) 
lower_bridge_BAND_7 = dados_filtrado['BAND_7'].quantile(0.25)-(IQR_BAND_7*1.5) 
upper_bridge_BAND_7 = dados_filtrado['BAND_7'].quantile(0.75)+(IQR_BAND_7*1.5)
lower_bridge_BAND_7, upper_bridge_BAND_7

IQR_BAND_8 = dados_filtrado.BAND_8.quantile(0.75)-dados_filtrado.BAND_8.quantile(0.25) 
lower_bridge_BAND_8 = dados_filtrado['BAND_8'].quantile(0.25)-(IQR_BAND_8*1.5) 
upper_bridge_BAND_8 = dados_filtrado['BAND_8'].quantile(0.75)+(IQR_BAND_8*1.5)
lower_bridge_BAND_8, upper_bridge_BAND_8

IQR_BAND_8A = dados_filtrado.BAND_8A.quantile(0.75)-dados_filtrado.BAND_8A.quantile(0.25) 
lower_bridge_BAND_8A = dados_filtrado['BAND_8A'].quantile(0.25)-(IQR_BAND_8A*1.5) 
upper_bridge_BAND_8A = dados_filtrado['BAND_8A'].quantile(0.75)+(IQR_BAND_8A*1.5)
lower_bridge_BAND_8A, upper_bridge_BAND_8A

IQR_BAND_9 = dados_filtrado.BAND_9.quantile(0.75)-dados_filtrado.BAND_9.quantile(0.25) 
lower_bridge_BAND_9 = dados_filtrado['BAND_9'].quantile(0.25)-(IQR_BAND_9*1.5) 
upper_bridge_BAND_9 = dados_filtrado['BAND_9'].quantile(0.75)+(IQR_BAND_9*1.5)
lower_bridge_BAND_9, upper_bridge_BAND_9

IQR_BAND_11 = dados_filtrado.BAND_11.quantile(0.75)-dados_filtrado.BAND_11.quantile(0.25) 
lower_bridge_BAND_11 = dados_filtrado['BAND_11'].quantile(0.25)-(IQR_BAND_11*1.5) 
upper_bridge_BAND_11 = dados_filtrado['BAND_11'].quantile(0.75)+(IQR_BAND_11*1.5)
lower_bridge_BAND_11, upper_bridge_BAND_11

IQR_BAND_12 = dados_filtrado.BAND_12.quantile(0.75)-dados_filtrado.BAND_12.quantile(0.25) 
lower_bridge_BAND_12 = dados_filtrado['BAND_12'].quantile(0.25)-(IQR_BAND_12*1.5) 
upper_bridge_BAND_12 = dados_filtrado['BAND_12'].quantile(0.75)+(IQR_BAND_12*1.5)
lower_bridge_BAND_12, upper_bridge_BAND_12

In [ ]:
dados_filtrado.describe()

In [ ]:
# Desvio Padrão - Determinação do Limite de Corte - Bandas

print('Verificação dos Limites de Corte das Bandas Espectrais:')
print(f'*Desvio Padrão da Banda_1: {dados_filtrado.BAND_1.std()}')
print(f'*Desvio Padrão da Banda_2: {dados_filtrado.BAND_2.std()}')
print(f'*Desvio Padrão da Banda_3: {dados_filtrado.BAND_3.std()}')
print(f'*Desvio Padrão da Banda_4: {dados_filtrado.BAND_4.std()}')
print(f'*Desvio Padrão da Banda_5: {dados_filtrado.BAND_5.std()}')
print(f'*Desvio Padrão da Banda_6: {dados_filtrado.BAND_6.std()}')
print(f'*Desvio Padrão da Banda_7: {dados_filtrado.BAND_7.std()}')
print(f'*Desvio Padrão da Banda_8: {dados_filtrado.BAND_8.std()}')
print(f'*Desvio Padrão da Banda_8A: {dados_filtrado.BAND_8A.std()}')
print(f'*Desvio Padrão da Banda_9: {dados_filtrado.BAND_9.std()}')
print(f'*Desvio Padrão da Banda_11: {dados_filtrado.BAND_11.std()}')
print(f'*Desvio Padrão da Banda_12: {dados_filtrado.BAND_12.std()}')



print('---------------------------------------------------------------------')


print('Para Prosseguir Sem Efetuar Filtros por Desvio Padrão: Digite 1: ')
print('Para Prosseguir Filtrar por Valores do Desvio Padrão: Digite 2:  ')
print('Para Prosseguir Filtrar por Valores InterQuartis: Digite 3:  ')

usar_todos_dados = int(input())

# --------------------------------------------

if usar_todos_dados==2:

    print('---------------------------------------------------------------------')
    print('Filtro de Bandas Realizado por Desvio Padrão: ')
    print('---------------------------------------------------------------------')
    print('Digite o Valor pelo qual o Desvio Padrão Deverá ser Multiplicado (inteiro): ')
    valor_multiplica_desvio_padrao = int(input())

    print(f'*O Valor Limite de Corte da Banda_1 é : {dados_filtrado.BAND_1.std() * valor_multiplica_desvio_padrao}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_1'] > dados_filtrado.BAND_1.std() * valor_multiplica_desvio_padrao].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print('Filtro de Bandas Realizado por Intervalos InterQuartis: ')
    print(f'*O Valor Limite de Corte da Banda_2 é : {dados_filtrado.BAND_2.std() * valor_multiplica_desvio_padrao}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_2'] > dados_filtrado.BAND_2.std() * valor_multiplica_desvio_padrao].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_3 é : {dados_filtrado.BAND_3.std() * valor_multiplica_desvio_padrao}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_3'] > dados_filtrado.BAND_3.std() * valor_multiplica_desvio_padrao].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_4 é : {dados_filtrado.BAND_4.std() * valor_multiplica_desvio_padrao}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_4'] > dados_filtrado.BAND_4.std() * valor_multiplica_desvio_padrao].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_5 é : {dados_filtrado.BAND_5.std() * valor_multiplica_desvio_padrao}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_5'] > dados_filtrado.BAND_5.std() * valor_multiplica_desvio_padrao].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_6 é : {dados_filtrado.BAND_6.std() * valor_multiplica_desvio_padrao}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_6'] > dados_filtrado.BAND_6.std() * valor_multiplica_desvio_padrao].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_7 é : {dados_filtrado.BAND_7.std() * valor_multiplica_desvio_padrao}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_7'] > dados_filtrado.BAND_7.std() * valor_multiplica_desvio_padrao].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_8 é : {dados_filtrado.BAND_8.std() * valor_multiplica_desvio_padrao}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_8'] > dados_filtrado.BAND_8.std() * valor_multiplica_desvio_padrao].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_8A é : {dados_filtrado.BAND_8A.std() * valor_multiplica_desvio_padrao}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_8A'] > dados_filtrado.BAND_8A.std() * valor_multiplica_desvio_padrao].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_9 é : {dados_filtrado.BAND_9.std() * valor_multiplica_desvio_padrao}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_9'] > dados_filtrado.BAND_9.std() * valor_multiplica_desvio_padrao].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_11 é : {dados_filtrado.BAND_11.std() * valor_multiplica_desvio_padrao}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_11'] > dados_filtrado.BAND_11.std() * valor_multiplica_desvio_padrao].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_12 é : {dados_filtrado.BAND_12.std() * valor_multiplica_desvio_padrao}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_12'] > dados_filtrado.BAND_12.std() * valor_multiplica_desvio_padrao].index, inplace = True)
    print('---------------------------------------------------------------------')

# ---------------------------------------------------
    
if usar_todos_dados==3:

    print('---------------------------------------------------------------------')
    print('Filtro de Bandas Realizado por Intervalos InterQuartis: ')
    print('---------------------------------------------------------------------')

    print(f'*O Valor Limite de Corte da Banda_1 é : {upper_bridge_BAND_1}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_1'] > upper_bridge_BAND_1].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print('Filtro de Bandas Realizado por Intervalos InterQuartis: ')
    print(f'*O Valor Limite de Corte da Banda_2 é : {upper_bridge_BAND_2}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_2'] > upper_bridge_BAND_2].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_3 é : {upper_bridge_BAND_3}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_3'] > upper_bridge_BAND_3].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_4 é : {upper_bridge_BAND_4}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_4'] > upper_bridge_BAND_4].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_5 é : {upper_bridge_BAND_5}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_5'] > upper_bridge_BAND_5].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_6 é : {upper_bridge_BAND_6}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_6'] > upper_bridge_BAND_6].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_7 é : {upper_bridge_BAND_7}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_7'] > upper_bridge_BAND_7].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_8 é : {upper_bridge_BAND_8}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_8'] > upper_bridge_BAND_8].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_8A é : {upper_bridge_BAND_8A}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_8A'] > upper_bridge_BAND_8A].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_9 é : {upper_bridge_BAND_9}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_9'] > upper_bridge_BAND_9].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_11 é : {upper_bridge_BAND_11}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_11'] > upper_bridge_BAND_11].index, inplace = True)
    print('---------------------------------------------------------------------')
    
    print(f'*O Valor Limite de Corte da Banda_12 é : {upper_bridge_BAND_12}')
    dados_filtrado.drop(dados_filtrado[dados_filtrado['BAND_12'] > upper_bridge_BAND_12].index, inplace = True)
    print('---------------------------------------------------------------------')

# ---------------------------------------
    
else:    
    dados_filtrado = dados_filtrado.copy()



#dados_filtrado.head()
print(f'*Quantidade de Dados Disponíveis Após Aplicação do Filtro por Desvio Padrão/InterQuartis : {dados_filtrado.shape[0]}')

In [ ]:
dados_filtrado.describe()

In [ ]:
# Desvio Padrão - Determinação do Limite de Corte - Variável de Interesse

print('Verificação do Limite de Corte da Variável de Interesse:')
print(f'*Desvio Padrão da Variável de Interesse: {dados_filtrado.VALUE.std()}')
print('Para Prosseguir Sem Efetuar Filtros por Desvio Padrão: Digite 1: ')
print('Para Prosseguir Filtrar por Valores do Desvio Padrão: Digite 2:  ')
print('Para Prosseguir Filtrar por Valores InterQuartis: Digite 3:  ')

usar_todos_dados = int(input())

if usar_todos_dados==2:
    print('Filtro por Desvio Padrão: ')
    print('Digite o Valor pelo qual o Desvio Padrão Deverá ser Multiplicado (inteiro): ')
    valor_multiplica_desvio_padrao = int(input())
    
    limite_corte_variavel_desvio_padrao = valor_multiplica_desvio_padrao * dados_filtrado.VALUE.std()
    
    print(f'*O Valor Limite de Corte é : {limite_corte_variavel_desvio_padrao}')

    dados_filtrado.drop(dados_filtrado[dados_filtrado['VALUE'] > limite_corte_variavel_desvio_padrao].index, inplace = True)

elif usar_todos_dados==3:
    print('Filtro por Intervalos InterQuartis: ')
    IQR=dados_filtrado.VALUE.quantile(0.75)-dados_filtrado.VALUE.quantile(0.25) 
    lower_bridge=dados_filtrado['VALUE'].quantile(0.25)-(IQR*1.5) 
    upper_bridge=dados_filtrado['VALUE'].quantile(0.75)+(IQR*1.5)
    
    print(f'*O Valor Limite de Corte é : {upper_bridge}')

    dados_filtrado.drop(dados_filtrado[dados_filtrado['VALUE'] > upper_bridge].index, inplace = True)
    
else:    
    dados_filtrado = dados_filtrado.copy()
    #valor_limite = 'Sem Limitacao'
    #diferenca_dias = 'Sem Limitacao'


#dados_filtrado.head()
print(f'*Quantidade de Dados Disponíveis Após Aplicação do Filtro por Desvio Padrão/InterQuartis : {dados_filtrado.shape[0]}')

In [ ]:
#dados_filtrado.info()

In [ ]:
# Nova verificação da Distribução após corte de valores de bandas
x = 'VALUE'
fig, ax = plt.subplots(nrows=1, ncols=2,  sharex=False, sharey=False)
fig.suptitle(x, fontsize=20)
### distribution
ax[0].title.set_text('distribution')
variable = dados_filtrado[x].fillna(dados_filtrado[x].mean())
breaks = np.quantile(variable, q=np.linspace(0, 1, 11))
variable = variable[ (variable > breaks[0]) & (variable < 
                    breaks[10]) ]
sns.distplot(variable, hist=True, kde=True, kde_kws={"shade": True}, ax=ax[0])
des = dados_filtrado[x].describe()
ax[0].axvline(des["25%"], ls='--')
ax[0].axvline(des["mean"], ls='--')
ax[0].axvline(des["75%"], ls='--')
ax[0].grid(True)
des = round(des, 2).apply(lambda x: str(x))
box = '\n'.join(("min: "+des["min"], "25%: "+des["25%"], "mean: "+des["mean"], "75%: "+des["75%"], "max: "+des["max"]))
ax[0].text(0.95, 0.95, box, transform=ax[0].transAxes, fontsize=10, va='top', ha="right", bbox=dict(boxstyle='round', facecolor='white', alpha=1))
### boxplot 
ax[1].title.set_text('outliers (log scale)')
tmp_dtf = pd.DataFrame(dados_filtrado[x])
#tmp_dtf[x] = np.log(dados_filtrado[x])
tmp_dtf.boxplot(column=x, ax=ax[1])
plt.show()

In [ ]:
# Selecionando as Bandas Espectrais de Interesse

In [ ]:
#Filtro de Bandas Espectrais de Interesse

print('Quais variáveis de entrada vc quer utlizar?')
print('Digite 1 para sim ou 0 para Não')
print('1: Selecionar as 12 bandas espectrais')
print('2: Selecionar TODAS as VARIÁVEIS DE ENTRADA disponíveis')
print('0: Informar as bandas de interesse')

usar_todas_bandas = int(input())
print('------------------------------------------------------')

if usar_todas_bandas==0:
    print('------------------------------------------------------')
    print('Digite uma lista com as bandas espectrais desejadas')
    print('Digite Todas: para selecionar todas as 12 bandas simultaneamente')
    print('Ou')
    print('Digite as bandas desejadas. Exemplo: band_1 band_2 band_3')
    print('------------------------------------------------------')
    bandas = (input()).upper()
    bandas = bandas.split()
    bandas.append('VALUE')
    #bandas, len(bandas)
    dados_filtrado1 = dados_filtrado[bandas]

    
if usar_todas_bandas == 1:
    dados_filtrado1 = dados_filtrado[['BAND_1','BAND_2','BAND_3','BAND_4','BAND_5','BAND_6','BAND_7','BAND_8','BAND_8A','BAND_9','BAND_11','BAND_12','VALUE']]
    bandas = dados_filtrado1.columns
    
    
else:    
    #turb_filtrado1 = turb_filtrado[turb_filtrado.columns]
    #dados_filtrado1 = dados_filtrado[['BAND_1','BAND_2','BAND_3','BAND_4','BAND_5','BAND_6','BAND_7','BAND_8','BAND_8A','BAND_9','BAND_11','BAND_12','VALUE']]
    
    #dados_filtrado1 = dados_filtrado[['BAND_1','BAND_2','BAND_3','BAND_4','BAND_5','BAND_6','BAND_7','BAND_8','BAND_8A','BAND_9','BAND_11','BAND_12', 'NDWI', 'SWI', 'VALUE']]
    
    #dados_filtrado1 = dados_filtrado[['BAND_1','BAND_2','BAND_3','BAND_4','BAND_5','BAND_6','BAND_7','BAND_8','BAND_8A','BAND_9','BAND_11','BAND_12', 'NDWI', 'M_NDWI','SWI', 'MCARI','VALUE']]
    
    dados_filtrado1 = dados_filtrado[['BAND_1','BAND_2','BAND_3','BAND_4','BAND_5','BAND_6','BAND_7','BAND_8','BAND_8A','BAND_9','BAND_11','BAND_12', 'NDWI', 'M_NDWI','SWI', 'MCARI', 'NDVI', 'NBSI', 'MOISTURE', 'WATER_BODIES','WATER_PLANTS', 'VALUE']]
    
    bandas = dados_filtrado1.columns
    

dados_filtrado1.head()


In [ ]:
# Separando bandas da variável dependente para filtros futuros
bandas = bandas[0:(len(bandas)-1)]
bandas

In [ ]:
# Ajustando Fator de Escala de Bandas Espectrais: 1/10.000

In [ ]:
lista_multiplicar_bandas = ['BAND_1','BAND_2','BAND_3','BAND_4','BAND_5','BAND_6','BAND_7','BAND_8','BAND_8A','BAND_9','BAND_11','BAND_12']

In [ ]:
dados_filtrado1[lista_multiplicar_bandas] = dados_filtrado1[lista_multiplicar_bandas].multiply(0.0001)

In [ ]:
dados_filtrado1

# Preparando para Gravar dados que serão utilizados nos modelos de IA
'''
arquivo_utilizado = str(nome_arquivo + '_' + str(len(dados_filtrado1)) + '_dados_' +
    str(valor_limite) + '_IntervaloDados_' +
    str(diferenca_dias) + '_DiferencaDias_' +
    str(len(bandas)) + '_bandas.xlsx')

#arquivo_utilizado

pasta_raiz = r'C:\Users\peixe\simoa_1'

#str(pasta_raiz)+'\\'+nome_arquivo


# Criar pasta
pasta_gravacao = str(pasta_raiz)+'\\'+nome_arquivo
if not os.path.exists(pasta_gravacao):
    os.makedirs(pasta_gravacao)
    

# Caminho do arquivo
caminho_arquivo = str(pasta_gravacao) +'\\'+ str(arquivo_utilizado)
#caminho_arquivo


# Gravar dados
dados_filtrado1.to_excel(caminho_arquivo)

# Limpa o nome do arquivo após a gravação
caminho_arquivo = ''
'''

In [ ]:
# PyWedge

In [ ]:
#!pip install pywedge

In [ ]:
#import pywedge as pw

In [ ]:
#mc = pw.Pywedge_Charts(dados_filtrado1, c=None, y="VALUE")
#chart = mc.make_charts()

In [ ]:
#from pandas_profiling import ProfileReport

In [ ]:
#profile = ProfileReport(dados_filtrado1, title="Pandas Profiling Report", explorative=True)

In [ ]:
#profile.to_notebook_iframe()

In [ ]:
#import sweetviz as sv
 
# generate report
#sweet_report = sv.analyze(dados_filtrado1)

In [ ]:
#sweet_report.show_html()

In [ ]:
#compare_report = sv.compare([y_test,"teste"], [predictions,"predicao"], "output")
 # show output within notebook file
#compare_report.show_html()

In [ ]:
#bandas =['BAND_1', 'BAND_3', 'BAND_5', 'BAND_11']
#bandas =['BAND_1', 'BAND_3', 'BAND_5']

In [ ]:
# Tempo de execução de Algoritmos

In [ ]:
# Verificar tempo - Início
tempo_inicial = time.time() # em segundos

In [ ]:
# Função Linear - Aplicação da OLS

In [ ]:
# Montando Função Linear - Parte 1

string_funcao_linear_part1 = ''

for x in bandas:
    string_funcao_linear_part1 += '+' + x

#print(string_funcao_linear_part1)

string_funcao_linear_part1 = string_funcao_linear_part1[1:len(string_funcao_linear_part1)]

#print(string_funcao_linear_part1)

#############################################

# Montando Função Linear - Parte 2

string_funcao_linear_part2 = 'VALUE ~ ' + string_funcao_linear_part1
print('Função Linear: ',string_funcao_linear_part2)

In [ ]:
# Regressão Linear - Função OLS

print('------------------------------------------------------------------------------')
print('--------------------------------Resultados------------------------------------')
print(f'*Valor Limite do parâmetro: {valor_limite} / *Diferença de Data: {diferenca_dias}')
print('------------------------------------------------------------------------------')

result = sm.ols(formula=string_funcao_linear_part2, data=dados_filtrado1).fit()
print(result.summary())

In [ ]:
## Função Linear - Verificaçaõ dos Resultados

In [ ]:
# Pegando os parâmetros da Função Linear OLS
result.params, result.params[0]

In [ ]:
# Pegando os valores dos parâmetros
var = result.params
var[1]
var

In [ ]:
# Separando variáveis dependentes e independentes
X1 = dados_filtrado1[bandas]
y1 = dados_filtrado1['VALUE']

In [ ]:
# Aplicando a Equação Linear
# novas colunas

if len(bandas) == 12:
    y_teste_ols1 = var[0] + var[1]*X1['BAND_1'] + var[2]*X1['BAND_2'] + var[3]*X1['BAND_3'] + var[4]*X1['BAND_4'] + var[5]*X1['BAND_5'] + var[6]*X1['BAND_6'] + var[7]*X1['BAND_7'] + var[8]*X1['BAND_8'] + var[9]*X1['BAND_8A'] + var[10]*X1['BAND_9'] + var[11]*X1['BAND_11'] + var[12]*X1['BAND_12']
else:
    y_teste_ols1 = var[0] + var[1]*X1['BAND_1'] + var[2]*X1['BAND_2'] + var[3]*X1['BAND_3'] + var[4]*X1['BAND_4'] + var[5]*X1['BAND_5'] + var[6]*X1['BAND_6'] + var[7]*X1['BAND_7'] + var[8]*X1['BAND_8'] + var[9]*X1['BAND_8A'] + var[10]*X1['BAND_9'] + var[11]*X1['BAND_11'] + var[12]*X1['BAND_12'] + var[13]*X1['NDWI'] + var[14]*X1['M_NDWI'] +var[15]*X1['SWI'] + var[16]*X1['MCARI'] + var[17]*X1['NDVI'] + var[18]*X1['NBSI'] + var[19]*X1['MOISTURE'] + var[20]*X1['WATER_BODIES'] + var[21]*X1['WATER_PLANTS']

    

# 21 bandas
#y_teste_ols1 = var[0] + var[1]*X1['BAND_1'] + var[2]*X1['BAND_2'] + var[3]*X1['BAND_3'] + var[4]*X1['BAND_4'] + var[5]*X1['BAND_5'] + var[6]*X1['BAND_6'] + var[7]*X1['BAND_7'] + var[8]*X1['BAND_8'] + var[9]*X1['BAND_8A'] + var[10]*X1['BAND_9'] + var[11]*X1['BAND_11'] + var[12]*X1['BAND_12'] + var[13]*X1['NDWI'] + var[14]*X1['M_NDWI'] +var[15]*X1['SWI'] + var[16]*X1['MCARI'] + var[17]*X1['NDVI'] + var[18]*X1['NBSI'] + var[19]*X1['MOISTURE'] + var[20]*X1['WATER_BODIES'] + var[21]*X1['WATER_PLANTS']


# y_teste_ols1 = var[0] + var[1]*X1['BAND_1'] + var[2]*X1['BAND_2'] + var[3]*X1['BAND_3'] + var[4]*X1['BAND_4'] + var[5]*X1['BAND_5'] + var[6]*X1['BAND_6'] + var[7]*X1['BAND_7'] + var[8]*X1['BAND_8'] + var[9]*X1['BAND_8A'] + var[10]*X1['BAND_9'] + var[11]*X1['BAND_11'] + var[12]*X1['BAND_12'] + var[13]*X1['NDWI'] + var[14]*X1['M_NDWI'] +var[15]*X1['SWI'] + var[16]*X1['MCARI']


#y_teste_ols1 = var[0] + var[1]*X1['BAND_1'] + var[2]*X1['BAND_2'] + var[3]*X1['BAND_3'] + var[4]*X1['BAND_4'] + var[5]*X1['BAND_5'] + var[6]*X1['BAND_6'] + var[7]*X1['BAND_7'] + var[8]*X1['BAND_8'] + var[9]*X1['BAND_8A'] + var[10]*X1['BAND_9'] + var[11]*X1['BAND_11'] + var[12]*X1['BAND_12'] + var[13]*X1['NDWI'] + var[14]*X1['SWI']

# 12 bandas
#y_teste_ols1 = var[0] + var[1]*X1['BAND_1'] + var[2]*X1['BAND_2'] + var[3]*X1['BAND_3'] + var[4]*X1['BAND_4'] + var[5]*X1['BAND_5'] + var[6]*X1['BAND_6'] + var[7]*X1['BAND_7'] + var[8]*X1['BAND_8'] + var[9]*X1['BAND_8A'] + var[10]*X1['BAND_9'] + var[11]*X1['BAND_11'] + var[12]*X1['BAND_12']

#y_teste_ols1 = var[0] + var[1]*X1['BAND_1'] + var[2]*X1['BAND_2'] + var[3]*X1['BAND_3'] + var[4]*X1['BAND_4'] + var[5]*X1['BAND_5'] + var[6]*X1['BAND_8A']

#y_teste_ols1 = var[0] + var[1]*X1['BAND_1'] + var[2]*X1['BAND_3'] + var[3]*X1['BAND_5'] # + var[4]*X1['BAND_11']


# Resultado da Aplicação da Função
y_teste_ols_previsto1 = y_teste_ols1.values
#y_teste_ols_previsto1

In [ ]:
# Valor original da variável idependente
y_teste_ols_verdadeiro1 = y1.values
#y_teste_ols_verdadeiro1

In [ ]:
# Regressão Linear
print('------------------------------------------------------------------------------')
print('--------------------------------Resultados------------------------------------')
print('Função Linear - OLS-----------------------------------------------------------')
metricas(y_teste_ols_verdadeiro1, y_teste_ols_previsto1)

In [ ]:
# Selecionar bandas
#bandas = bandas_importantes

In [ ]:
# Codigo do Tipo de Algoritmo
# 1- Linear
# 2 - Árvore de Decisão
# 3- Agrupamento
# 4- Gradient Boosting
# 5- Rede Neural

In [ ]:
# Selecionando variáveis independentes
#global X
#X = dados_filtrado1[bandas].values
#X
#X.shape

In [ ]:
def seleciona_dados_variaveis_indepentes_x(codigo_tipo_algoritmo):
    global X
    global y
    global bandas_utilizadas
    if codigo_tipo_algoritmo in(1, 2, 3, 4):
        X = dados_filtrado1[bandas_mais_frequentes].values
        print(X.shape)
        
        y = dados_filtrado1['VALUE'].values
        print(y.shape)
        
        bandas_utilizadas = bandas_mais_frequentes
    
    else:
        X = dados_filtrado1[bandas].values
        print(X.shape)
        
        y = dados_filtrado1['VALUE'].values
        print(y.shape)
        
        bandas_utilizadas = bandas
    
    return X, y

In [ ]:
# Settando as variáveis X e y com todos os dados
seleciona_dados_variaveis_indepentes_x(5)
clear_output()

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# Registro de Resultados - Preparação da Tabela

In [ ]:
# Tamanho das Amostras de Treino e Teste - Cross-Validation
samples_cv = X.shape[0]

X_test_cv_tamanho = np.ceil(X.shape[0]/5).astype(int)
X_train_cv_tamanho = samples_cv - X_test_cv_tamanho

train_test_cv = str(str(X_train_cv_tamanho) + '/' + str(X_test_cv_tamanho))

#samples_cv, train_test_cv

In [ ]:
# Preparando a tabela para gravação

In [ ]:
results_frame = pd.DataFrame()
columns_result = ['Method','Samples','Train/Test', 'Split Mode', 'R2', 'MSE', 'RMSE', 'MAE', 'MAPE']

In [ ]:
results_frame = pd.DataFrame(columns=columns_result)
results_frame

In [ ]:
#esvaziar frame (caso haja valores guardados)
results_frame = results_frame[0:0]

In [ ]:
# Cross-Validation

In [ ]:
# Divisiões do Conjunto de Dados
folds_number = 5

In [ ]:
random_fixo = 42

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [ ]:
## APLICAÇÃO DOS MODELOS DE INTELIGÊNCIA ARTIFICIAL ##

In [ ]:
## Modelos de Inteligência Artificial ##

In [ ]:
# Regressão Linear

In [ ]:
# Nome do método utilizado
method = 'Linear Regression'

In [ ]:
# Selecionando principais bandas para o o tipo de algoritmo
seleciona_dados_variaveis_indepentes_x(5)
#

scores = []
R2_lista, MSE_lista, RMSE_lista, MAE_lista, MAPE_lista = [], [], [], [], []

#method = results_frame['Method'].iloc[0]

linear_regression = LinearRegression()

cv = KFold(n_splits=folds_number, random_state=random_fixo, shuffle=True)

melhor_r2 = 0

for train_index, test_index in cv.split(X):
    #print("Train Index: ", train_index, "\n")
    #print("Test Index: ", test_index, "\n")
    print("Metrics")

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    linear_regression.fit(X_train, y_train)
    scores.append(linear_regression.score(X_test, y_test))
    
    predictions = linear_regression.predict(X_test)
    
    metricas(y_test, predictions)
    
    R2_lista.append(R2)
    MSE_lista.append(MSE)
    RMSE_lista.append(RMSE)
    MAE_lista.append(MAE)
    MAPE_lista.append(MAPE)

    
    if R2 > melhor_r2:
        melhor_r2 = R2
        
        melhor_X_train = sc.fit_transform(X_train)
        melhor_y_train = y_train
        melhor_X_test = sc.transform(X_test)
        melhor_y_test = y_test
    
    #print("---------------------------------------")
    clear_output()

In [ ]:
R2 = statistics.mean(R2_lista)
MSE = statistics.mean(MSE_lista)
RMSE = statistics.mean(RMSE_lista)
MAE = statistics.mean(MAE_lista)
MAPE = statistics.mean(MAPE_lista)

#R2, MSE, RMSE, MAE, MAPE

print('Method: ', method)
print("R2 = ", R2)
print("MSE = ", MSE)
print("RMSE = ", RMSE)
print("MAE = ", MAE)
print("MAPE = ", MAPE)

In [ ]:
# Exbindo Resultados
# Cross-Validation

division = 'Cross-Validation'

results_frame = results_frame.append({'Method': method,
                                      'Samples': samples_cv,
                                      'Train/Test': train_test_cv,
                                      'Split Mode': division,
                                      'R2': R2,
                                      'MSE': MSE,
                                      'RMSE': RMSE,
                                      'MAE': MAE,
                                      'MAPE': MAPE},
                                     ignore_index=True)

results_frame

In [ ]:
# Redes Neurais

In [ ]:
taxa_aprendizado = 0.01

In [ ]:
epocas = 1000

In [ ]:
n_batchs = 128

In [ ]:
callback = keras.callbacks.EarlyStopping(
    monitor='val_mae', patience=50, verbose=2, mode='min',
    baseline=None, restore_best_weights=True)

In [ ]:
# Neural Comum

In [ ]:
# method = 'ANN 5L, 100N'

In [ ]:
method = 'ANN 5L, 100N'

In [ ]:
# Selecionando principais bandas para o o tipo de algoritmo
#seleciona_dados_variaveis_indepentes_x(1)
seleciona_dados_variaveis_indepentes_x(5)
clear_output()
#

In [ ]:
modelo_5L_100 = Sequential()
modelo_5L_100.add(Dense(units = 100, activation = 'relu', input_shape=(len(bandas_utilizadas),)))
#modelo_5L_100.add(Dropout(0.2))
modelo_5L_100.add(Dense(units = 100, activation = 'relu'))
#modelo_5L_100.add(Dropout(0.2))
modelo_5L_100.add(Dense(units = 100, activation = 'relu'))
#modelo_5L_100.add(Dropout(0.2))
modelo_5L_100.add(Dense(units = 100, activation = 'relu'))
#modelo_5L_100.add(Dropout(0.2))
modelo_5L_100.add(Dense(units = 100, activation = 'relu'))
#modelo_5L_100.add(Dropout(0.2))
modelo_5L_100.add(Dense(1))

In [ ]:
Optimizer = tf.keras.optimizers.Adam(learning_rate = taxa_aprendizado)

In [ ]:
modelo_5L_100.compile(Optimizer, loss='mse', metrics=["mae","mse"])

In [ ]:
#scores = []
R2_lista, MSE_lista, RMSE_lista, MAE_lista, MAPE_lista = [], [], [], [], []

#method = results_frame['Method'].iloc[0]

cv = KFold(n_splits=folds_number, random_state=random_fixo, shuffle=True)

melhor_r2 = 0

for train_index, test_index in cv.split(X):
    #print("Train Index: ", train_index, "\n")
    #print("Test Index: ", test_index, "\n")
    print("Metrics")

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    
    
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    
    history_5L_100 = modelo_5L_100.fit(X_train , y_train , validation_data=(X_test , y_test) ,
                           callbacks=[callback],
                           epochs=epocas, batch_size=n_batchs)
    
    predictions = modelo_5L_100.predict(X_test)
    
    metricas(y_test, predictions)
    
    R2_lista.append(R2)
    MSE_lista.append(MSE)
    RMSE_lista.append(RMSE)
    MAE_lista.append(MAE)
    MAPE_lista.append(MAPE)

    
    if R2 > melhor_r2:
        melhor_r2 = R2
        
        melhor_X_train = sc.fit_transform(X_train)
        melhor_y_train = y_train
        melhor_X_test = sc.transform(X_test)
        melhor_y_test = y_test
    
    #print("---------------------------------------")
    clear_output()

In [ ]:
R2 = statistics.mean(R2_lista)
MSE = statistics.mean(MSE_lista)
RMSE = statistics.mean(RMSE_lista)
MAE = statistics.mean(MAE_lista)
MAPE = statistics.mean(MAPE_lista)

#R2, MSE, RMSE, MAE, MAPE

print('Method: ', method)
print("R2 = ", R2)
print("MSE = ", MSE)
print("RMSE = ", RMSE)
print("MAE = ", MAE)
print("MAPE = ", MAPE)

In [ ]:
# Exbindo Resultados
# Cross-Validation

division = 'Cross-Validation'

results_frame = results_frame.append({'Method': method,
                                      'Samples': samples_cv,
                                      'Train/Test': train_test_cv,
                                      'Split Mode': division,
                                      'R2': R2,
                                      'MSE': MSE,
                                      'RMSE': RMSE,
                                      'MAE': MAE,
                                      'MAPE': MAPE},
                                     ignore_index=True)

results_frame

In [ ]:
import sys
'geopandas' in sys.modules
!pip install geopandas

In [3]:
import pandas as pd
import geopandas as gpd

In [4]:
dados = gpd.read_file('4_Converter_Shape/B01.shp')
dados.head()

,VALUE,geometry
0,0.03430,"POLYGON ((466540.000 7988920.000, 466540.000 7..."
1,0.04265,"POLYGON ((466560.000 7988920.000, 466560.000 7..."
2,0.05100,"POLYGON ((466580.000 7988920.000, 466580.000 7..."
3,0.04170,"POLYGON ((466540.000 7988900.000, 466540.000 7..."
4,0.04045,"POLYGON ((466560.000 7988900.000, 466560.000 7..."


In [5]:
dados.shape

(2132649, 2)

In [6]:
dados.rename(columns = {'VALUE': 'BAND_1'},
                      inplace=True)
dados.head()

,BAND_1,geometry
0,0.03430,"POLYGON ((466540.000 7988920.000, 466540.000 7..."
1,0.04265,"POLYGON ((466560.000 7988920.000, 466560.000 7..."
2,0.05100,"POLYGON ((466580.000 7988920.000, 466580.000 7..."
3,0.04170,"POLYGON ((466540.000 7988900.000, 466540.000 7..."
4,0.04045,"POLYGON ((466560.000 7988900.000, 466560.000 7..."


In [ ]:
#b1 = gpd.read_file('B01.shp')
#dados['BAND_1'] = b1['VALUE']
#b1.head()

In [7]:
b2 = gpd.read_file('4_Converter_Shape/B02.shp')
dados['BAND_2'] = b2['VALUE']
del(b2)
dados.head()

,BAND_1,geometry,BAND_2
0,0.03430,"POLYGON ((466540.000 7988920.000, 466540.000 7...",0.032150
1,0.04265,"POLYGON ((466560.000 7988920.000, 466560.000 7...",0.028700
2,0.05100,"POLYGON ((466580.000 7988920.000, 466580.000 7...",0.029925
3,0.04170,"POLYGON ((466540.000 7988900.000, 466540.000 7...",0.052525
4,0.04045,"POLYGON ((466560.000 7988900.000, 466560.000 7...",0.029100


In [8]:
b3 = gpd.read_file('4_Converter_Shape/B03.shp')
dados['BAND_3'] = b3['VALUE']
del(b3)
dados.head()

,BAND_1,geometry,BAND_2,BAND_3
0,0.03430,"POLYGON ((466540.000 7988920.000, 466540.000 7...",0.032150,0.043875
1,0.04265,"POLYGON ((466560.000 7988920.000, 466560.000 7...",0.028700,0.036975
2,0.05100,"POLYGON ((466580.000 7988920.000, 466580.000 7...",0.029925,0.040300
3,0.04170,"POLYGON ((466540.000 7988900.000, 466540.000 7...",0.052525,0.084300
4,0.04045,"POLYGON ((466560.000 7988900.000, 466560.000 7...",0.029100,0.038600


In [9]:
b4 = gpd.read_file('4_Converter_Shape/B04.shp')
dados['BAND_4'] = b4['VALUE']
del(b4)
dados.head()

,BAND_1,geometry,BAND_2,BAND_3,BAND_4
0,0.03430,"POLYGON ((466540.000 7988920.000, 466540.000 7...",0.032150,0.043875,0.049050
1,0.04265,"POLYGON ((466560.000 7988920.000, 466560.000 7...",0.028700,0.036975,0.031375
2,0.05100,"POLYGON ((466580.000 7988920.000, 466580.000 7...",0.029925,0.040300,0.037175
3,0.04170,"POLYGON ((466540.000 7988900.000, 466540.000 7...",0.052525,0.084300,0.114975
4,0.04045,"POLYGON ((466560.000 7988900.000, 466560.000 7...",0.029100,0.038600,0.035350


In [10]:
b5 = gpd.read_file('4_Converter_Shape/B05.shp')
dados['BAND_5'] = b5['VALUE']
del(b5)
dados.head()

,BAND_1,geometry,BAND_2,BAND_3,BAND_4,BAND_5
0,0.03430,"POLYGON ((466540.000 7988920.000, 466540.000 7...",0.032150,0.043875,0.049050,0.0619
1,0.04265,"POLYGON ((466560.000 7988920.000, 466560.000 7...",0.028700,0.036975,0.031375,0.0261
2,0.05100,"POLYGON ((466580.000 7988920.000, 466580.000 7...",0.029925,0.040300,0.037175,0.0339
3,0.04170,"POLYGON ((466540.000 7988900.000, 466540.000 7...",0.052525,0.084300,0.114975,0.1464
4,0.04045,"POLYGON ((466560.000 7988900.000, 466560.000 7...",0.029100,0.038600,0.035350,0.0376


In [11]:
b6 = gpd.read_file('4_Converter_Shape/B06.shp')
dados['BAND_6'] = b6['VALUE']
del(b6)
dados.head()

,BAND_1,geometry,BAND_2,BAND_3,BAND_4,BAND_5,BAND_6
0,0.03430,"POLYGON ((466540.000 7988920.000, 466540.000 7...",0.032150,0.043875,0.049050,0.0619,0.0584
1,0.04265,"POLYGON ((466560.000 7988920.000, 466560.000 7...",0.028700,0.036975,0.031375,0.0261,0.0239
2,0.05100,"POLYGON ((466580.000 7988920.000, 466580.000 7...",0.029925,0.040300,0.037175,0.0339,0.0418
3,0.04170,"POLYGON ((466540.000 7988900.000, 466540.000 7...",0.052525,0.084300,0.114975,0.1464,0.1525
4,0.04045,"POLYGON ((466560.000 7988900.000, 466560.000 7...",0.029100,0.038600,0.035350,0.0376,0.0364


In [12]:
b7 = gpd.read_file('4_Converter_Shape/B07.shp')
dados['BAND_7'] = b7['VALUE']
del(b7)
dados.head()

,BAND_1,geometry,BAND_2,BAND_3,BAND_4,BAND_5,BAND_6,BAND_7
0,0.03430,"POLYGON ((466540.000 7988920.000, 466540.000 7...",0.032150,0.043875,0.049050,0.0619,0.0584,0.0684
1,0.04265,"POLYGON ((466560.000 7988920.000, 466560.000 7...",0.028700,0.036975,0.031375,0.0261,0.0239,0.0302
2,0.05100,"POLYGON ((466580.000 7988920.000, 466580.000 7...",0.029925,0.040300,0.037175,0.0339,0.0418,0.0467
3,0.04170,"POLYGON ((466540.000 7988900.000, 466540.000 7...",0.052525,0.084300,0.114975,0.1464,0.1525,0.1747
4,0.04045,"POLYGON ((466560.000 7988900.000, 466560.000 7...",0.029100,0.038600,0.035350,0.0376,0.0364,0.0350


In [13]:
b8 = gpd.read_file('4_Converter_Shape/B08.shp')
dados['BAND_8'] = b8['VALUE']
del(b8)
dados.head()

,BAND_1,geometry,BAND_2,BAND_3,BAND_4,BAND_5,BAND_6,BAND_7,BAND_8
0,0.03430,"POLYGON ((466540.000 7988920.000, 466540.000 7...",0.032150,0.043875,0.049050,0.0619,0.0584,0.0684,0.061325
1,0.04265,"POLYGON ((466560.000 7988920.000, 466560.000 7...",0.028700,0.036975,0.031375,0.0261,0.0239,0.0302,0.032975
2,0.05100,"POLYGON ((466580.000 7988920.000, 466580.000 7...",0.029925,0.040300,0.037175,0.0339,0.0418,0.0467,0.038350
3,0.04170,"POLYGON ((466540.000 7988900.000, 466540.000 7...",0.052525,0.084300,0.114975,0.1464,0.1525,0.1747,0.164450
4,0.04045,"POLYGON ((466560.000 7988900.000, 466560.000 7...",0.029100,0.038600,0.035350,0.0376,0.0364,0.0350,0.036700


In [14]:
b8a = gpd.read_file('4_Converter_Shape/B08A.shp')
dados['BAND_8A'] = b8a['VALUE']
del(b8a)
dados.head()

,BAND_1,geometry,BAND_2,BAND_3,BAND_4,BAND_5,BAND_6,BAND_7,BAND_8,BAND_8A
0,0.03430,"POLYGON ((466540.000 7988920.000, 466540.000 7...",0.032150,0.043875,0.049050,0.0619,0.0584,0.0684,0.061325,0.0673
1,0.04265,"POLYGON ((466560.000 7988920.000, 466560.000 7...",0.028700,0.036975,0.031375,0.0261,0.0239,0.0302,0.032975,0.0338
2,0.05100,"POLYGON ((466580.000 7988920.000, 466580.000 7...",0.029925,0.040300,0.037175,0.0339,0.0418,0.0467,0.038350,0.0477
3,0.04170,"POLYGON ((466540.000 7988900.000, 466540.000 7...",0.052525,0.084300,0.114975,0.1464,0.1525,0.1747,0.164450,0.1762
4,0.04045,"POLYGON ((466560.000 7988900.000, 466560.000 7...",0.029100,0.038600,0.035350,0.0376,0.0364,0.0350,0.036700,0.0446


In [15]:
b9 = gpd.read_file('4_Converter_Shape/B09.shp')
dados['BAND_9'] = b9['VALUE']
del(b9)
dados.head()

,BAND_1,geometry,BAND_2,BAND_3,BAND_4,BAND_5,BAND_6,BAND_7,BAND_8,BAND_8A,BAND_9
0,0.03430,"POLYGON ((466540.000 7988920.000, 466540.000 7...",0.032150,0.043875,0.049050,0.0619,0.0584,0.0684,0.061325,0.0673,0.04000
1,0.04265,"POLYGON ((466560.000 7988920.000, 466560.000 7...",0.028700,0.036975,0.031375,0.0261,0.0239,0.0302,0.032975,0.0338,0.05295
2,0.05100,"POLYGON ((466580.000 7988920.000, 466580.000 7...",0.029925,0.040300,0.037175,0.0339,0.0418,0.0467,0.038350,0.0477,0.06590
3,0.04170,"POLYGON ((466540.000 7988900.000, 466540.000 7...",0.052525,0.084300,0.114975,0.1464,0.1525,0.1747,0.164450,0.1762,0.04470
4,0.04045,"POLYGON ((466560.000 7988900.000, 466560.000 7...",0.029100,0.038600,0.035350,0.0376,0.0364,0.0350,0.036700,0.0446,0.04310


In [16]:
b11 = gpd.read_file('4_Converter_Shape/B11.shp')
dados['BAND_11'] = b11['VALUE']
del(b11)
dados.head()

,BAND_1,geometry,BAND_2,BAND_3,BAND_4,BAND_5,BAND_6,BAND_7,BAND_8,BAND_8A,BAND_9,BAND_11
0,0.03430,"POLYGON ((466540.000 7988920.000, 466540.000 7...",0.032150,0.043875,0.049050,0.0619,0.0584,0.0684,0.061325,0.0673,0.04000,0.0957
1,0.04265,"POLYGON ((466560.000 7988920.000, 466560.000 7...",0.028700,0.036975,0.031375,0.0261,0.0239,0.0302,0.032975,0.0338,0.05295,0.0438
2,0.05100,"POLYGON ((466580.000 7988920.000, 466580.000 7...",0.029925,0.040300,0.037175,0.0339,0.0418,0.0467,0.038350,0.0477,0.06590,0.0675
3,0.04170,"POLYGON ((466540.000 7988900.000, 466540.000 7...",0.052525,0.084300,0.114975,0.1464,0.1525,0.1747,0.164450,0.1762,0.04470,0.2004
4,0.04045,"POLYGON ((466560.000 7988900.000, 466560.000 7...",0.029100,0.038600,0.035350,0.0376,0.0364,0.0350,0.036700,0.0446,0.04310,0.0638


In [17]:
b12 = gpd.read_file('4_Converter_Shape/B12.shp')
dados['BAND_12'] = b12['VALUE']
del(b12)
dados.head()

,BAND_1,geometry,BAND_2,BAND_3,BAND_4,BAND_5,BAND_6,BAND_7,BAND_8,BAND_8A,BAND_9,BAND_11,BAND_12
0,0.03430,"POLYGON ((466540.000 7988920.000, 466540.000 7...",0.032150,0.043875,0.049050,0.0619,0.0584,0.0684,0.061325,0.0673,0.04000,0.0957,0.0593
1,0.04265,"POLYGON ((466560.000 7988920.000, 466560.000 7...",0.028700,0.036975,0.031375,0.0261,0.0239,0.0302,0.032975,0.0338,0.05295,0.0438,0.0334
2,0.05100,"POLYGON ((466580.000 7988920.000, 466580.000 7...",0.029925,0.040300,0.037175,0.0339,0.0418,0.0467,0.038350,0.0477,0.06590,0.0675,0.0507
3,0.04170,"POLYGON ((466540.000 7988900.000, 466540.000 7...",0.052525,0.084300,0.114975,0.1464,0.1525,0.1747,0.164450,0.1762,0.04470,0.2004,0.1349
4,0.04045,"POLYGON ((466560.000 7988900.000, 466560.000 7...",0.029100,0.038600,0.035350,0.0376,0.0364,0.0350,0.036700,0.0446,0.04310,0.0638,0.0415


In [18]:
colunas_base = ['geometry', 'BAND_1','BAND_2','BAND_3','BAND_4','BAND_5','BAND_6','BAND_7','BAND_8','BAND_8A','BAND_9','BAND_11','BAND_12']
dados = dados[colunas_base]
dados.head()

,geometry,BAND_1,BAND_2,BAND_3,BAND_4,BAND_5,BAND_6,BAND_7,BAND_8,BAND_8A,BAND_9,BAND_11,BAND_12
0,"POLYGON ((466540.000 7988920.000, 466540.000 7...",0.03430,0.032150,0.043875,0.049050,0.0619,0.0584,0.0684,0.061325,0.0673,0.04000,0.0957,0.0593
1,"POLYGON ((466560.000 7988920.000, 466560.000 7...",0.04265,0.028700,0.036975,0.031375,0.0261,0.0239,0.0302,0.032975,0.0338,0.05295,0.0438,0.0334
2,"POLYGON ((466580.000 7988920.000, 466580.000 7...",0.05100,0.029925,0.040300,0.037175,0.0339,0.0418,0.0467,0.038350,0.0477,0.06590,0.0675,0.0507
3,"POLYGON ((466540.000 7988900.000, 466540.000 7...",0.04170,0.052525,0.084300,0.114975,0.1464,0.1525,0.1747,0.164450,0.1762,0.04470,0.2004,0.1349
4,"POLYGON ((466560.000 7988900.000, 466560.000 7...",0.04045,0.029100,0.038600,0.035350,0.0376,0.0364,0.0350,0.036700,0.0446,0.04310,0.0638,0.0415


In [19]:
# codigo bruno
dados.to_pickle("221011_mapa_baixa_turbidez.pkl")

In [ ]:
#testar load do arquivo salvo
unpickled_df = pd.read_pickle("mapa.pkl")
unpickled_df

In [ ]:
colunas_interesse = colunas_base = ['BAND_1','BAND_2','BAND_3','BAND_4','BAND_5','BAND_6','BAND_7','BAND_8','BAND_8A','BAND_9','BAND_11','BAND_12']

In [ ]:
X_atributos = dados[colunas_interesse].values

In [ ]:
X_atributos

In [ ]:
X_atributos = sc.transform(X_atributos)
X_atributos

In [ ]:
y_previsoes_atributos = modelo_5L_100.predict(X_atributos)

In [ ]:
#y_previsoes_atributos = linear_regression.predict(X_atributos)

In [ ]:
y_previsoes_atributos.shape

In [ ]:
atributos_previstos = dados.copy()
del(dados)
atributos_previstos.head()

In [ ]:
atributos_previstos['variavel'] = y_previsoes_atributos
atributos_previstos.head()

In [ ]:
atributos_previstos_1 = atributos_previstos[['geometry', 'variavel']]
del(atributos_previstos)
atributos_previstos_1.head()

In [ ]:
atributos_previstos_1.shape

In [ ]:
atributos_previstos_1['variavel'].describe()

In [ ]:
atributos_previstos_1['variavel'].value_counts()

In [ ]:
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10,15

In [ ]:
fig, ax = plt.subplots(1, 1)
atributos_previstos_1.plot(column='variavel', legend = True, vmin=0.0, vmax=3, ax=ax)
ax.set_title(variavel_interesse + ' - Modelagem: Reservatório de Três Marias - 21/01/2021')

In [ ]:
type(atributos_previstos_1)